# **CS490/590: HW2 - Backpropagation and Classification**


---


## Due Date 
**March 8, 2022**

*Total 15 points*

---

## **Background**: #
The goal of this homework is for you to implement backpropagation on a classification problem that is not linear separable. This will provide you a sense of how neural networks are implemented from scratch with gradient descent and backpropogation. Specifically, you will be implementing a multilayer perceptron from formulas found in your lectures and notes. 


In [1]:
# load relevant libraries
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
import math
from random import seed
from random import random

%matplotlib inline

## Create the toy dataset
The dataset is simply two concentric circles.

In [99]:
np.random.seed(0)

def make_dataset(num_points):
    radius = 5
    data = []
    labels = np.zeros((num_points, 2))
    cnt = 0
    # Generate positive examples (labeled 1).
    for i in range(num_points // 2):
        r = np.random.uniform(0, radius*0.5)
        angle = np.random.uniform(0, 2*math.pi)
        x = r * math.sin(angle)
        y = r * math.cos(angle)
        data.append([x, y])
        labels[cnt,0] = 1
        cnt = cnt + 1
        
    # Generate negative examples (labeled 0).
    for i in range(num_points // 2):
        r = np.random.uniform(radius*0.7, radius)
        angle = np.random.uniform(0, 2*math.pi)
        x = r * math.sin(angle)
        y = r * math.cos(angle)
        data.append([x, y])
        labels[cnt,1] = 1
        cnt = cnt + 1
        
    data = np.asarray(data)
    labels = np.asarray(labels)
    return data, labels
    
num_data = 1000
data, labels = make_dataset(num_data)
# Note: red indicates a label of 1, blue indicates a label of 0
plt.scatter(data[:num_data//2, 0], data[:num_data//2, 1], color='red') 
plt.scatter(data[num_data//2:, 0], data[num_data//2:, 1], color='blue')     

In [4]:
data.shape

---
## **Question 1:** Defining the neural network *(6 points)*


Using a neural network, we will classifiy the above data. The input will be a two dimensional vector $\mathbf{x} \in \mathbb{R}^{2 \times 1} $ giving the coordinates of a point in 2-D space. The output is a vector $\mathbf{y} \in \mathbb{R}^{2}$ containing the probability $P(y = t | \mathbf{x})$ where $t = 1,2$ indicates the point $\mathbf{x}$ is in circle 1 or 2.  

A neural network with two hidden layers which have 30 and 20 hidden units each, will be used. The equations describing our neural network for a single observtion are given below:

$$\mathbf{z}^{(1)} = \mathbf{W}^{(1)} \mathbf{x} + \mathbf{b}^{(1)}$$
$$\mathbf{h}^{(1)} = \tanh(\mathbf{z}^{(1)})$$
$$\mathbf{z}^{(2)} = \mathbf{W}^{(2)} \mathbf{h}^{(1)} + \mathbf{b}^{(2)}$$
$$\mathbf{h}^{(2)} = \tanh(\mathbf{z}^{(2)})$$
$$\mathbf{z}^{(3)} = \mathbf{W}^{(3)} \mathbf{h}^{(2)} + \mathbf{b}^{(3)}$$
$$\mathbf{y} = \sigma(\mathbf{z}^{(3)})$$

where, $\mathbf{W}^{(1)}\in\mathbb{R}^{30 \times 2}, \mathbf{b}^{(1)}\in\mathbb{R}^{30}, \mathbf{W}^{(2)}\in\mathbb{R}^{20 \times 30}, \mathbf{b}^{(2)}\in\mathbb{R}^{20}, \mathbf{W}^{(3)}\in\mathbb{R}^{2 \times 20}, \mathbf{b}^{(3)}  \in \mathbb{R}^{2}$ are the parameters of our neural network which we must learn and $\sigma$ is the softmax function.



### Vectorizing the forward pass of the neural network

To compute the predictions more efficiently we need to vectorize over the training examples. Let  $\mathbf{X} \in \mathbb{R}^{N  \times 2}$ be a matrix containing $N$ observations in separate rows. Then we can vectorize the above formulas by:

$$\mathbf{Z}^{(1)} = \mathbf{X}{\mathbf{W}^{(1)}}^T + \mathbf{1}{\mathbf{b}^{(1)}}^T$$
$$\mathbf{H}^{(1)} = \tanh(\mathbf{Z}^{(1)})$$
$$\mathbf{Z}^{(2)} = \mathbf{H}^{(1)}{\mathbf{W}^{(2)}}^T + \mathbf{1}{\mathbf{b}^{(2)}}^T$$
$$\mathbf{H}^{(2)} = \tanh(\mathbf{Z}^{(2)})$$
$$\mathbf{z} = \mathbf{H}^{(2)}{\mathbf{W}^{(3)}}^T + \mathbf{1}b^{(3)}$$
$$\mathbf{y} = \sigma(\mathbf{z})$$


In the code below, fill in the formulas, and initialize the weights with values from the uniform random distribiution and the biases with 0's.

In [115]:
# FIRST INITIALIZE THE NEURAL NETWORK PARAMETERS.
#
# Make use of numpy functions to do the initializatons. 
# Make sure you follow the correct dimensions for vectors and matrices. 


### YOUR CODE HERE ###
input_layer=2
hidden_1=30
hidden_2=20
output_layer=2
params = {
        'W1':np.random.randn(hidden_1, input_layer) * np.sqrt(1./hidden_1),
        'W2':np.random.randn(hidden_2, hidden_1) * np.sqrt(1./hidden_2),
        'W3':np.random.randn(output_layer, hidden_2) * np.sqrt(1./output_layer),
    'b1':0,
    'b2':0,
    'b3':0
    }
cache = {}
######################
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        print(np.asarray(weights).shape,np.asarray(inputs).shape)
        activation += weights[i] * inputs[i]+biases[i]
        print(activation)
    return activation


# Hyperbolic Function
def activation_hyperbolic(x,derivative=False):
    if derivative:
        t=(np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        dt=1-t**2
        return dt
    else:
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

def activation_sigmoid(x):
    return 1 / (1 + np.exp(-x))
# DEFINE THE FORWARD PASS OF THE NETWORK.
# 
# You can use params['W1'] and the other parameters like regular numpy variables. 
def forward(x,params):
    cache["X"] = x
    cache["Z1"] = np.matmul(params["W1"], cache["X"].T) + params["b1"]
    cache["A1"] = activation_hyperbolic(cache["Z1"])
    cache["Z2"] = np.matmul(params["W2"], cache["A1"]) + params["b2"]
    cache["A2"] = activation_hyperbolic(cache["Z2"])
    cache["Z3"] = np.matmul(params["W3"], cache["A2"]) + params["b3"]
    cache["A3"] = activation_sigmoid(cache["Z3"])
    return cache["A3"]



In [116]:
print(params)

### Visualize the network's predictions

Let's visualize the predictions of our untrained network. As we can see, the network does not succeed at classifying the points without training

In [117]:
num_points = 200
x1s = np.linspace(-6.0, 6.0, num_points)
x2s = np.linspace(-6.0, 6.0, num_points)

points = np.transpose([np.tile(x1s, len(x2s)), np.repeat(x2s, len(x1s))])
Y = forward(points,params)
Y = Y.transpose()

Yp = np.argmax(Y , axis=1) # To get the actual prediction label we need to find the column with maximum probability. 
Yp = Yp.reshape(num_points, num_points) # Reshape the predicted points into a 2D grid.
X1, X2 = np.meshgrid(x1s, x2s)

plt.pcolormesh(X1, X2, Yp, cmap=plt.cm.get_cmap('YlGn'))
plt.colorbar()
plt.scatter(data[:num_data//2, 0], data[:num_data//2, 1], color='red') 
plt.scatter(data[num_data//2:, 0], data[num_data//2:, 1], color='blue') 

---
## **Question 2:** Implementing backpropogation *(9 points)*
### Loss function

We will use the same cross entropy loss function as in logistic regression. This loss function is:

$$\mathcal{L}_{CE}(y, t) = -t \log(y) - (1 - t)\log(1 - y)$$

Here $y = P(t = 1|\mathbf{x})$ and $t$ is the true label.

Remember that computing the derivative of this loss function $\frac{d L}{dy}$ can become numerically unstable. Instead, we combine the logistic function and the cross entropy loss into a single function called logistic cross-entropy:

$$\mathcal{L}_{LCE}(z, t) = t \log(1 + \exp(-z)) + (1 -t) \log(1 + \exp(z))$$

See Lecture Notes for a review. 

Our cost function is the sum over multiple examples of the loss function, normalized by the number of examples:

$$\mathcal{E}(\mathbf{z}, \mathbf{t}) = \frac{1}{N} \left[\sum_{i=1}^N \mathcal{L}(z_i, t_i)\right]$$

### Derive and implement the backpropagation equations

Derive the backpropagation equations in vector form and provide the code below. Rememember that the derivates of the matrices and vectors should have the same dimensions as the orignal variables. Take a look at the lecture notes and slides and on how to do backpropagation for MLPs and to find the derivative for softmax-cross-entropy. 


In [118]:
def backprop(y, t,params):
    
    N = y.shape[0]

    # PERFORM THE FORWARD COMPUTATION
    #
    # same as from above in Q.1
    ### YOUR CODE HERE ### 
   
#     cache={}
#     cache["X"] = X
#     cache["Z1"] = np.matmul(params["W1"], cache["X"].T) + params["b1"]
#     cache["A1"] = activation_hyperbolic(cache["Z1"])
#     cache["Z2"] = np.matmul(params["W2"], cache["A1"]) + params["b2"]
#     cache["A2"] = activation_hyperbolic(cache["Z2"])
#     cache["Z3"] = np.matmul(params["W3"], cache["A2"]) + params["b3"]
#     cache["A3"] = activation_sigmoid(cache["Z3"])
#     y = cache["A3"]
    
#     loss = (1./N) * np.sum(-t * np.log(y) - (1 - t) * np.log(1 - y))
    

    # PERFORM THE BACKWARD COPMUTATION.
    #
    ### YOUR CODE HERE ###
    
    current_batch_size = y.shape[0]
    dZ3 = t - y.T
    
    W3_bar = (1./current_batch_size) * np.matmul(dZ3, cache["A2"].T)
    b3_bar = (1./current_batch_size) * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.matmul(params["W3"].T, dZ3)
    dZ2 = dA2 * activation_hyperbolic(cache["Z2"], derivative=True)
    W2_bar = (1./current_batch_size) * np.matmul(dZ2, cache["A1"].T)
    b2_bar = (1./current_batch_size) * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.matmul(params["W2"].T, dZ2)
    dZ1 = dA1 * activation_hyperbolic(cache["Z1"], derivative=True)
    W1_bar = (1./current_batch_size) * np.matmul(dZ1, cache["X"])
    b1_bar = (1./current_batch_size) * np.sum(dZ1, axis=1, keepdims=True)

    # Wrap our gradients in a dictionary.
    grads = {}
    grads['W1'] = W1_bar
    grads['b1'] = b1_bar
    grads['W2'] = W2_bar
    grads['b2'] = b2_bar
    grads['W3'] = W3_bar
    grads['b3'] = b3_bar
    print(grads)
    
    return grads,dZ3

## Training the network

Train the network parameters with gradient descent using the derivatives from the backpropagation algorithm. Recall that the gradient descent update rule for a given parameter $\mathbf{w}^{(i)}$ in the $i^{th}$ layer and learning rate $\alpha$ is:

$$\mathbf{w}^{(i)} \gets \mathbf{w}^{(i)} - \alpha * \frac{\partial \mathcal{E}}{\partial \mathbf{w}^{(i)}}$$

Also make sure that you choose the correct $\alpha$ and the correct number of steps, to reach s loss of almost 0. 

In [120]:

            
def grad_descent(X, t, params, print_every, niter, alpha):

    for i in range(niter):

        # Forward
        output = forward(X,params)
        # Backprop
        grad,loss = backprop(t, output,params)
        # Optimize
        params = optimize(alpha, 0.9,grad)
        
        if i % print_every == 0:
            print(i)
#             print("Step {:3d} | Loss {:3.2f}".format(i, loss))
    
    return params

def optimize(l_rate, beta,grads):
    
    '''
        Stochatic Gradient Descent (SGD):
        θ^(t+1) <- θ^t - η∇L(y, ŷ)

     
    '''
    optimizer = 'sgd'
    
    if optimizer == "sgd":
        for key in params:
            params[key] = params[key] - l_rate*grads[key]
        return params
num_steps = 10000
alpha = 0.1


params = grad_descent(data, labels, params, 10, num_steps, alpha)


In [121]:
print(params)

## Visualizing the predictions
Now when we visualize the prediction we see that the correct labels are highlighted.

In [122]:
num_points = 200
x1s = np.linspace(-6.0, 6.0, num_points)
x2s = np.linspace(-6.0, 6.0, num_points)

points = np.transpose([np.tile(x1s, len(x2s)), np.repeat(x2s, len(x1s))])
Y = forward(points, params)
Y = Y.transpose()
Yp = np.argmax(Y, axis=1)
Yp = Yp.reshape(num_points, num_points)
X1, X2 = np.meshgrid(x1s, x2s)

plt.pcolormesh(X1, X2, Yp, cmap=plt.cm.get_cmap('YlGn'))
plt.colorbar()
plt.scatter(data[:num_data//2, 0], data[:num_data//2, 1], color='red') 
plt.scatter(data[num_data//2:, 0], data[num_data//2:, 1], color='blue') 